In [8]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from ipywidgets import interact

%matplotlib inline

In [9]:
def fix_dataframe(df):
    params = []
    vals = []
    for key, val in df.items():
        if key == "Unnamed: 0":
            continue
        params.append(key)
        vals.append(val)

    dicos = []
    for stuff in zip(*vals):
        others = []
        dico = {}
        for param, val in zip(params, stuff):
            if not param.startswith("eps"):
                dico[param] = val
            else:
                others.append((param, val))

        if not isinstance(dico["embedding_type"], str):
            dico["method"] = dico["binary"].upper()
        else:
            dico["method"] = dico["embedding_type"]
        for param, val in others:
            sup, eps = param.split("_")[1:3]
            eps = float(eps)
            dicos.append({"eps": eps, "AUC": val, "supervised": sup == "s", **dico})

    return pd.DataFrame(dicos)

df = pd.read_csv("csv/results_8_04_2020_readable.csv")
df = fix_dataframe(df)
df = df.loc[df.architecture != "svhn_lenet_bandw2"]

In [11]:
def filter_column(df, column, value):
    return df.loc[np.logical_or(df[column].isnull(), df[column] == value)]

@interact
def plot_results(# architecture=df.architecture.unique(),
                 supervised=[True, False],
                 mlp=[False, True]
                 # epochs=[50, 200, 250, 300]
                 ):
    subdf = df.copy()
    subdf["mlp"] = subdf["architecture"].apply(lambda arch: "mlp" in arch or "simple_fcn" in arch)
    for param, val in zip(["supervised", "mlp"],
                          [supervised, mlp]):
        subdf = filter_column(subdf, param, val)
    sns.catplot(data=subdf, x="eps", y="AUC", hue="method", kind="point", col="architecture", col_wrap=2)
    plt.tight_layout()
    plt.savefig("/home/elvis/Downloads/fig_sup=%s_mlp=%s.png" % (supervised, mlp),
                dpi=200, bbox_inches="tight")

interactive(children=(Dropdown(description='supervised', options=(True, False), value=True), Dropdown(descript…